In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Import

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # reduce TF log noise (optional)

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print("TF:", tf.__version__)
print("TF-DF:", tfdf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))


2026-01-11 12:50:10.459847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768135810.676904      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768135810.737260      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768135811.201132      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768135811.201176      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768135811.201178      24 computation_placer.cc:177] computation placer alr

TF: 2.19.0
TF-DF: 1.12.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load Dataset

In [3]:
DATA_DIR = "/kaggle/input/house-prices-advanced-regression-techniques"

train_raw = pd.read_csv(f"{DATA_DIR}/train.csv")
test_raw  = pd.read_csv(f"{DATA_DIR}/test.csv")

print(train_raw.shape, test_raw.shape)
train_raw.head()

(1460, 81) (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Split (IDs + target)

In [4]:
train_ids = train_raw["Id"].copy()
test_ids  = test_raw["Id"].copy()

y = train_raw["SalePrice"].copy()

X_train_raw = train_raw.drop(columns=["Id", "SalePrice"])
X_test_raw  = test_raw.drop(columns=["Id"])

print(X_train_raw.shape, X_test_raw.shape)

(1460, 79) (1459, 79)


# Preprocess

TF-DF can ingest categoricals directly (no need for one-hot). We’ll just fill missing values.

In [5]:
full = pd.concat([X_train_raw, X_test_raw], axis=0, ignore_index=True)

num_cols = full.select_dtypes(include=["int64", "float64"]).columns
cat_cols = full.select_dtypes(include=["object"]).columns

# Fill numeric with median
full[num_cols] = full[num_cols].fillna(full[num_cols].median())

# Fill categorical with "None"
full[cat_cols] = full[cat_cols].fillna("None")

# Split back
X_train = full.iloc[:len(X_train_raw)].copy()
X_test  = full.iloc[len(X_train_raw):].copy()

# Add target back (log transform is standard for Kaggle House Prices)
train_df = X_train.copy()
train_df["SalePrice"] = np.log1p(y)

test_df = X_test.copy()

print(train_df.shape, test_df.shape)
train_df.head()

(1460, 80) (1459, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,12.247699
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,12.109016
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,12.317171
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,11.849405
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,12.429220


# Feature Engineering

In [6]:
for df in [X_train, X_test]:
    df["TotalSF"] = (
        df["TotalBsmtSF"]
        + df["1stFlrSF"]
        + df["2ndFlrSF"]
    )

    df["TotalBathrooms"] = (
        df["FullBath"]
        + 0.5 * df["HalfBath"]
        + df["BsmtFullBath"]
        + 0.5 * df["BsmtHalfBath"]
    )

    df["HouseAge"] = df["YrSold"] - df["YearBuilt"]
    df["RemodAge"] = df["YrSold"] - df["YearRemodAdd"]

# Convert To TF-DF (REGRESSION)

This is the key part that fixes the “too many classes” error.

In [7]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    train_df,
    label="SalePrice",
    task=tfdf.keras.Task.REGRESSION
)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_df,
    task=tfdf.keras.Task.REGRESSION
)

I0000 00:00:1768135826.682282      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


# Gradient Boost Tree Model

Train model using gradient boost tree

In [8]:
model = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.REGRESSION,
    num_trees=3000,
    max_depth=5,
    min_examples=5,
    subsample=0.9,
    shrinkage=0.03,
    random_seed=42
)


model.fit(train_ds)
model.summary()

Use /tmp/tmpa2zoph7h as temporary training directory
Reading training dataset...


W0000 00:00:1768135826.931604      24 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1768135826.932151      24 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1768135826.932157      24 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:04.229158. Found 1460 examples.
Training model...


I0000 00:00:1768135831.194654      24 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1768135831.194691      24 kernel.cc:783] Collect training examples
I0000 00:00:1768135831.194699      24 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1768135831.195418      24 kernel.cc:401] Number of batches: 2
I0000 00:00:1768135831.195435      24 kernel.cc:402] Number of examples: 1460
I0000 00:00:1768135831.197779      24 data_spec_inference.cc:354] 1 item(s) have been pruned (i.e. they are considered out of dictionary) for the column BsmtCond (4 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:1768135831.197830      24 data_spec_inference.cc:354] 1 item(s) have been prun

Model trained in 0:00:03.075735
Compiling model...


I0000 00:00:1768135834.141429      82 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.125839
I0000 00:00:1768135834.141481      82 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 325
I0000 00:00:1768135834.150199      82 gradient_boosted_trees.cc:279] Truncates the model to 296 tree(s) i.e. 296  iteration(s).
I0000 00:00:1768135834.150432      82 gradient_boosted_trees.cc:341] Final model num-trees:296 valid-loss:0.125839 valid-rmse:0.125839
I0000 00:00:1768135834.156712      82 kernel.cc:926] Export model in log directory: /tmp/tmpa2zoph7h with prefix 26b2aab5b0044b65
I0000 00:00:1768135834.163942      82 kernel.cc:944] Save model in resources
I0000 00:00:1768135834.168483      24 abstract_model.cc:921] Model self evaluation:
Number of predictions (with weights): 1
Task: REGRESSION
Loss (SQUARED_ERROR): 0.125839

RMSE: 0.354737
Default RMSE: : 0

I0000 00:00:1768135834.261176    

Model compiled.
Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: REGRESSION
Label: "__LABEL"

Input Features (79):
	1stFlrSF
	2ndFlrSF
	3SsnPorch
	Alley
	BedroomAbvGr
	BldgType
	BsmtCond
	BsmtExposure
	BsmtFinSF1
	BsmtFinSF2
	BsmtFinType1
	BsmtFinType2
	BsmtFullBath
	BsmtHalfBath
	BsmtQual
	BsmtUnfSF
	CentralAir
	Condition1
	Condition2
	Electrical
	EnclosedPorch
	ExterCond
	ExterQual
	Exterior1st
	Exterior2nd
	Fence
	FireplaceQu
	Fireplaces
	Foundation
	FullBath
	Functional
	GarageArea
	GarageCars
	GarageCond
	GarageFinish
	GarageQual
	GarageType
	GarageYrBlt
	GrLivArea
	HalfBath
	Heating
	HeatingQC
	HouseStyle
	KitchenAbvGr
	KitchenQual
	LandContour
	LandSlope
	LotArea
	

In [9]:
gbt2 = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.REGRESSION,
    num_trees=3500,
    max_depth=4,
    shrinkage=0.03,
    subsample=0.9,
    min_examples=5,
    random_seed=7
)
gbt2.fit(train_ds)

Use /tmp/tmpz78k8nvu as temporary training directory
Reading training dataset...


W0000 00:00:1768135835.791884      24 gradient_boosted_trees.cc:1873] "goss_alpha" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1768135835.791920      24 gradient_boosted_trees.cc:1883] "goss_beta" set but "sampling_method" not equal to "GOSS".
W0000 00:00:1768135835.791923      24 gradient_boosted_trees.cc:1897] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training dataset read in 0:00:00.732419. Found 1460 examples.
Training model...


I0000 00:00:1768135836.535924      24 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1768135836.535962      24 kernel.cc:783] Collect training examples
I0000 00:00:1768135836.535973      24 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1768135836.536575      24 kernel.cc:401] Number of batches: 2
I0000 00:00:1768135836.536599      24 kernel.cc:402] Number of examples: 1460
I0000 00:00:1768135836.539011      24 data_spec_inference.cc:354] 1 item(s) have been pruned (i.e. they are considered out of dictionary) for the column BsmtCond (4 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:1768135836.539069      24 data_spec_inference.cc:354] 1 item(s) have been prun

Model trained in 0:00:02.450644
Compiling model...


I0000 00:00:1768135838.883563    1411 early_stopping.cc:54] Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.111024
I0000 00:00:1768135838.883613    1411 gradient_boosted_trees.cc:1669] Create final snapshot of the model at iteration 469
I0000 00:00:1768135838.890564    1411 gradient_boosted_trees.cc:279] Truncates the model to 440 tree(s) i.e. 440  iteration(s).
I0000 00:00:1768135838.890710    1411 gradient_boosted_trees.cc:341] Final model num-trees:440 valid-loss:0.111024 valid-rmse:0.111024
I0000 00:00:1768135838.895349    1411 kernel.cc:926] Export model in log directory: /tmp/tmpz78k8nvu with prefix b53a486b1400493d
I0000 00:00:1768135838.900297    1411 kernel.cc:944] Save model in resources
I0000 00:00:1768135838.902507      24 abstract_model.cc:921] Model self evaluation:
Number of predictions (with weights): 1
Task: REGRESSION
Loss (SQUARED_ERROR): 0.111024

RMSE: 0.333202
Default RMSE: : 0



Model compiled.


# Ensemble

In [10]:
rf_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,
    num_trees=1500,
    max_depth=18,
    random_seed=42
)

rf_model.fit(train_ds)
rf_model.summary()

Use /tmp/tmpa5y6v3rl as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.710324. Found 1460 examples.
Training model...


I0000 00:00:1768135840.313364      24 kernel.cc:782] Start Yggdrasil model training
I0000 00:00:1768135840.313398      24 kernel.cc:783] Collect training examples
I0000 00:00:1768135840.313407      24 kernel.cc:795] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

I0000 00:00:1768135840.313725      24 kernel.cc:401] Number of batches: 2
I0000 00:00:1768135840.313749      24 kernel.cc:402] Number of examples: 1460
I0000 00:00:1768135840.316068      24 data_spec_inference.cc:354] 1 item(s) have been pruned (i.e. they are considered out of dictionary) for the column BsmtCond (4 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
I0000 00:00:1768135840.316107      24 data_spec_inference.cc:354] 1 item(s) have been prun

Model trained in 0:00:11.629887
Compiling model...


I0000 00:00:1768135851.804660      24 decision_forest.cc:808] Model loaded with 1500 root(s), 696608 node(s), and 78 input feature(s).
I0000 00:00:1768135851.804722      24 abstract_model.cc:1439] Engine "RandomForestOptPred" built


Model compiled.
Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (79):
	1stFlrSF
	2ndFlrSF
	3SsnPorch
	Alley
	BedroomAbvGr
	BldgType
	BsmtCond
	BsmtExposure
	BsmtFinSF1
	BsmtFinSF2
	BsmtFinType1
	BsmtFinType2
	BsmtFullBath
	BsmtHalfBath
	BsmtQual
	BsmtUnfSF
	CentralAir
	Condition1
	Condition2
	Electrical
	EnclosedPorch
	ExterCond
	ExterQual
	Exterior1st
	Exterior2nd
	Fence
	FireplaceQu
	Fireplaces
	Foundation
	FullBath
	Functional
	GarageArea
	GarageCars
	GarageCond
	GarageFinish
	GarageQual
	GarageType
	GarageYrBlt
	GrLivArea
	HalfBath
	Heating
	HeatingQC
	HouseStyle
	KitchenAbvGr
	KitchenQual
	LandContour
	LandSlope
	LotArea
	LotConfig
	LotFron

# Inspect Feature Importances

In [11]:
inspector = model.make_inspector()
importances = inspector.variable_importances()

# "NUM_AS_ROOT" is usually a nice quick look
if "NUM_AS_ROOT" in importances:
    df_imp = pd.DataFrame(importances["NUM_AS_ROOT"])
    display(df_imp.head(20))
else:
    print(importances.keys())

,0,1
0,"(OverallQual, 1, 62)",35.0
1,"(GrLivArea, 1, 38)",30.0
2,"(OverallCond, 1, 61)",18.0
3,"(WoodDeckSF, 1, 75)",16.0
4,"(LotArea, 1, 47)",15.0
5,"(KitchenQual, 4, 44)",13.0
6,"(LotConfig, 4, 48)",12.0
7,"(YearRemodAdd, 1, 77)",12.0
8,"(Neighborhood, 4, 59)",11.0
9,"(BsmtUnfSF, 1, 15)",10.0


# Predict + Invert Log Transform

In [12]:
p1 = model.predict(test_ds).reshape(-1)
p2 = gbt2.predict(test_ds).reshape(-1)
p3 = rf_model.predict(test_ds).reshape(-1)

blend_log = 0.5*p1 + 0.4*p2 + 0.1*p3
pred = np.expm1(blend_log)

2/2 [==============================] - 0s 11ms/step


2/2 [==============================] - 0s 9ms/step


2/2 [==============================] - 0s 106ms/step


# Save For Submission

In [13]:
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": pred
})

submission.to_csv("submission.csv", index=False)
submission.to_parquet("submission.parquet", index=False)

print("Saved: submission.csv and submission.parquet")
submission.head()

Saved: submission.csv and submission.parquet


,Id,SalePrice
0,1461,124924.601562
1,1462,157908.609375
2,1463,184636.218750
3,1464,185845.984375
4,1465,197386.390625
